In [1]:
%mkdir -p /content/drive/MyDrive/GSE221321
%cd /content/drive/MyDrive/GSE221321

/content/drive/MyDrive/GSE221321


In [2]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE221321&format=file" -O GSE221321_RAW.tar
!tar -tvf GSE221321_RAW.tar

--2024-07-28 02:10:46--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE221321&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6260828160 (5.8G) [application/x-tar]
Saving to: ‘GSE221321_RAW.tar’

GSE221321_RAW.tar   100%[===================>]   5.83G  19.3MB/s    in 3m 14s  

2024-07-28 02:14:00 (30.8 MB/s) - ‘GSE221321_RAW.tar’ saved [6260828160/6260828160]

GSM6858447_KO_all_FRPerturb_effect_sizes.csv.gz
GSM6858447_KO_conventional.h5ad.gz
GSM6858447_KO_conventional.rds.gz
GSM6858447_KO_conventional_FRPerturb_effect_sizes.csv.gz
GSM6858447_KO_conventional_perturbations.txt.gz
GSM6858448_KO_cell_pooled.h5ad.gz
GSM6858448_KO_cell_pooled.rds.gz
GSM6858448_KO_cell_pooled_FRPerturb_effect_sizes.csv.gz
GSM6858448_KO_cell_pooled_perturbations.txt.gz
GSM6858449_KD_all_FRPerturb_effect_s

In [18]:
!tar -tvf GSE221321_RAW.tar | awk '{printf "%-20s\t%-20s\n", $3/1024/1024 " Mb", $6}'

60.944 Mb           	GSM6858447_KO_all_FRPerturb_effect_sizes.csv.gz
1015.25 Mb          	GSM6858447_KO_conventional.h5ad.gz
1030.25 Mb          	GSM6858447_KO_conventional.rds.gz
60.677 Mb           	GSM6858447_KO_conventional_FRPerturb_effect_sizes.csv.gz
0.713629 Mb         	GSM6858447_KO_conventional_perturbations.txt.gz
331.749 Mb          	GSM6858448_KO_cell_pooled.h5ad.gz
337.68 Mb           	GSM6858448_KO_cell_pooled.rds.gz
51.0628 Mb          	GSM6858448_KO_cell_pooled_FRPerturb_effect_sizes.csv.gz
0.308682 Mb         	GSM6858448_KO_cell_pooled_perturbations.txt.gz
64.3709 Mb          	GSM6858449_KD_all_FRPerturb_effect_sizes.csv.gz
1033.12 Mb          	GSM6858449_KD_conventional.h5ad.gz
1074.7 Mb           	GSM6858449_KD_conventional.rds.gz
64.7693 Mb          	GSM6858449_KD_conventional_FRPerturb_effect_sizes.csv.gz
0.5625 Mb           	GSM6858449_KD_conventional_perturbations.txt.gz
384.887 Mb          	GSM6858450_KD_guide_pooled.h5ad.gz
404.987 Mb          	GSM6858450_KD_g

In [19]:
!tar -tvf GSE221321_RAW.tar | grep -v '\.rds' | awk '{print $6}' | xargs -I {} tar -xvf GSE221321_RAW.tar {}
!gzip -d *.h5ad.gz

GSM6858447_KO_all_FRPerturb_effect_sizes.csv.gz
GSM6858447_KO_conventional.h5ad.gz
GSM6858447_KO_conventional_FRPerturb_effect_sizes.csv.gz
GSM6858447_KO_conventional_perturbations.txt.gz
GSM6858448_KO_cell_pooled.h5ad.gz
GSM6858448_KO_cell_pooled_FRPerturb_effect_sizes.csv.gz
GSM6858448_KO_cell_pooled_perturbations.txt.gz
GSM6858449_KD_all_FRPerturb_effect_sizes.csv.gz
GSM6858449_KD_conventional.h5ad.gz
GSM6858449_KD_conventional_FRPerturb_effect_sizes.csv.gz
GSM6858449_KD_conventional_perturbations.txt.gz
GSM6858450_KD_guide_pooled.h5ad.gz
GSM6858450_KD_guide_pooled_FRPerturb_effect_sizes.csv.gz
GSM6858450_KD_guide_pooled_perturbations.txt.gz


In [2]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.9 MB/s eta 0:00:00


In [12]:
adatas = []
for file in os.listdir():
    if file.endswith('.h5ad'):
        adata = sc.read_h5ad(file)
        adata.var.set_index('_index', inplace=True)
        adata.var.index.name = 'index'
        adata.obs['perturbation_name'] = adata.obs['Guides_collapsed_by_gene'].apply(
            lambda x: '+'.join(x.split('--'))
        )
        adata.obs['condition'] = adata.obs['perturbation_name'].apply(
            lambda x: x if x in ['safe-targeting', 'non-targeting'] else 'targeting'
        )
        adata.obs['n_guides'] = adata.obs['Total_number_of_guides'].astype(int)
        adata.obs['crispr_type'] = 'CRISPRi' if 'KD' in file else 'CRISPR KO'
        adata.obs['cancer_type'] = 'Leukemia'
        adata.obs['cell_type'] = 'THP-1 cell'
        adata.obs['organism'] = ORGANISMS[0]
        adatas.append(adata)
adata = ad.concat(adatas)

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata.obs.head()

,Total_RNA_count,Total_unique_genes,10X_channel,Percent_mitochondrial_reads,Guides,Guides_collapsed_by_gene,Total_number_of_guides,S_score,G2M_score,Cell_cycle_phase
AAACCCAAGAGTGACC-1,2413.0,1278,Normal_1,5.719022,TNFRSF11A_TTTGTGGCACTGGATCAATG_CRISPRko--STK4-...,TNFRSF11A--STK4-AS1,2.0,-0.072268,-0.068220,G1
AAACCCACACAGGATG-1,9174.0,2666,Normal_1,5.646392,GSDMC_GGAGCATCCATGGTCCACAG_CRISPRko--LRP1B_TGC...,GSDMC--LRP1B,2.0,-0.091449,-0.078500,G1
AAACCCACACCTCTAC-1,3779.0,1864,Normal_1,6.086266,NFATC2_GCGGAGGCATTCGTGCGCCG_CRISPRko--TAB3_AAA...,NFATC2--TAB3,2.0,0.341332,0.082301,S
AAACCCACATTGCCGG-1,9882.0,2892,Normal_1,4.604331,TXNDC17_CTGAACCAGTCGTACGAGAG_CRISPRko,TXNDC17,1.0,0.034257,-0.086085,S
AAACCCAGTCATCACA-1,10399.0,2514,Normal_1,3.413790,TNFSF11_CGTGGCTCGGAGGAGATGGG_CRISPRko,TNFSF11,1.0,-0.012602,-0.122855,G1


In [15]:
adata.obs['perturbation_name'].value_counts()

perturbation_name
non-targeting                                          9000
safe-targeting                                         6915
NFIC                                                    381
XKR6                                                    373
TNFRSF14                                                352
                                                       ... 
IER3+B4GALT1                                              1
IER3+BACH2+EGR2                                           1
IER3+BTK                                                  1
IER3+BUD13                                                1
safe-targeting+safe-targeting+safe-targeting+CTNND1       1
Name: count, Length: 41310, dtype: int64

In [18]:
adata.obs['condition'] = adata.obs['perturbation_name'].apply(
    lambda x: x if x in ['safe-targeting', 'non-targeting'] else 'targeting'
)
adata.obs['condition'].value_counts()

condition
targeting         194216
non-targeting       9000
safe-targeting      6915
Name: count, dtype: int64

In [13]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [19]:
adata.write_h5ad('GSE221321_merged.h5ad')